# **MySQL Set up**

In [6]:
import os
from pyspark.sql import SparkSession, SQLContext
from pyspark.sql.functions import col, when, length
from pyspark.sql import SparkSession

#data is stored in "/labs" locally
project_dir = "jupyter_workspace"

# You can explore the different folders using 
# `os.listdir()`
drivers = os.listdir(f"{project_dir}")

# MySQL 8 requires a specific driver
spark = SparkSession.builder \
        .config("spark.jars", "mysql-connector-j-8.3.0.jar") \
        .master("local").appName("PySpark_MySQL_test").getOrCreate()
# create url to connect to MySQL
# we use the service here "db" rather than the localhost
# we're also using the 3306 port since this is container to container
# to connect SQL ide we'll use the 13306 defined in our compose
url = "jdbc:mysql://db:3306/db"

# More connection props
# for local only "PLEASE never use root and password in the real world"
properties = {
    "user": "root",
    "password": "password",
    "driver": "com.mysql.jdbc.Driver"
}
# table name variable
# this will need to be loaded via the /csv in the labs
table_name = "2024_nfl_prospects"

# read the table
df = spark.read.jdbc(url, table_name, properties=properties)

# show the top 10 records
df.show(10,False)


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----------------+---------------+-----+---------+-----+---------+--------+-------+------+------+----+----+-----+-----+----+-----+----+-------+
|Player           |School         |Class|Birthdate|Years|Final Age|Position|Combine|Height|Weight|Hand|Arm |Forty|Bench|Vert|Broad|Cone|Shuttle|
+-----------------+---------------+-----+---------+-----+---------+--------+-------+------+------+----+----+-----+-----+----+-----+----+-------+
|Braelon Allen    |Wisconsin      |2021 |1/20/2004|3    |20.0     |RB      |1      |73.3  |235   |9.3 |31.3|     |26   |32.0|117  |    |       |
|Audric Estime    |Notre Dame     |2021 |9/6/2003 |3    |20.3     |RB      |1      |71.4  |221   |10.3|32.4|4.71 |23   |38.0|125  |    |       |
|Malik Nabers     |Louisiana State|2021 |7/28/2003|3    |20.4     |WR      |1      |      |      |    |    |     |     |    |     |    |       |
|Jonathon Brooks  |Texas          |2021 |7/21/2003|3    |20.5     |RB      |1      |72.4  |216   |9.3 |31.5|     |     |    |     

## **Add Column and Query Results**

In [3]:
# Create a DataFrame called df1

# Create a new column in the Dataframe called "is_fast"
# using the "Forty" Column add the values accordingly based on the times
    # "" = No Time
    # < 4.6 = faster
    # between 4.6, 5.2 = fast
    # All Others = slow

df1 = df.withColumn(
    "is_fast",
    when(length("Forty")== 0, "No Time")
    .when(col("Forty") < 4.6, "faster")
    .when(col("Forty").between(4.61, 5.2), "fast")
    .otherwise("slow")
)

# Show only participants with faster in the new column. 
df1.where(col("is_fast").isin(["faster"])).show()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------------+-----+----------+-----+---------+--------+-------+------+------+----+----+-----+-----+----+-----+----+-------+-------+
|          Player|             School|Class| Birthdate|Years|Final Age|Position|Combine|Height|Weight|Hand| Arm|Forty|Bench|Vert|Broad|Cone|Shuttle|is_fast|
+----------------+-------------------+-----+----------+-----+---------+--------+-------+------+------+----+----+-----+-----+----+-----+----+-------+-------+
|   Xavier Worthy|              Texas| 2021| 4/27/2003|    3|     20.7|      WR|      1|  71.3|   165| 8.8|31.1| 4.21|     |41.0|  131|    |       | faster|
|   Jaylen Wright|          Tennessee| 2021|  4/1/2003|    3|     20.8|      RB|      1|  70.5|   210| 9.4|31.5| 4.38|     |38.0|  134|    |       | faster|
|   Troy Franklin|             Oregon| 2021|  2/6/2003|    3|     20.9|      WR|      1|  73.9|   183| 8.6|31.9| 4.41|     |39.0|  124|6.90|   4.31| faster|
| Adonai Mitchell|              Texas| 2021| 10/8/2002|   